In [4]:
import numpy as np

In [9]:
"""Etire le vecteur v pour obtenir une matrice de taille n*taille_de_v dont chaque ligne est une copie de v"""
def stretch_vect(v, n):
    return [v for i in range(n)]

"""renvoie le produit des matrices m1 et m2 (terme à terme)"""
def prod_mat(m1, m2):
    n, m = m1.shape
    res = np.zeros(n, m)
    for i in range(n):
        for j in range(m):
            res[i][j] = m1[i][j]*m2[i][j]
    return res

"""renvoie la division des matrices m1 et m2 (terme à terme)"""
def div_mat(m1, m2):
    n, m = m1.shape
    res = np.zeros(n, m)
    for i in range(n):
        for j in range(m):
            res[i][j] = m1[i][j]/m2[i][j]
    return res

# 1. Mon premier est ... linéaire !

In [32]:

class Loss(object):
    def forward(self, y, yhat):
        pass

    def backward(self, y, yhat):
        pass

class MSELoss(Loss):
    def forward(self, y, yhat):
        return np.mean(np.power(y-yhat, 2), axis=1)

    def backward(self, y, yhat):
        return 2*(yhat-y)/len(y)


class Module(object):
    def __init__(self):
        self._parameters = None
        self._gradient = None

    def zero_grad(self):
        ## Annule gradient
        pass

    def forward(self, X):
        ## Calcule la passe forward
        pass

    def update_parameters(self, gradient_step=1e-3):
        ## Calcule la mise a jour des parametres selon le gradient calcule et le pas de gradient_step
        self._parameters -= gradient_step*self._gradient

    def backward_update_gradient(self, input, delta):
        ## Met a jour la valeur du gradient
        pass

    def backward_delta(self, input, delta):
        ## Calcul la derivee de l'erreur
        pass
    
class LinearModule(Module):
    """
    N : la taille du mini-batch
    d : le nombre de dimensions d'un échantillon
    d' : le nombre de neurones dans le module"""
    def __init__(self, d, dprime, N):
        Module.__init__(self)
        self.W = np.random.rand(d, dprime) #W est de taille d*d'
        self.b = stretch_vect(np.random.rand(1, dprime), N) #b est de taille N*d' (on réplique N fois la première ligne)
        self.gradientW = None
        self.gradientb = None

    """def zero_grad(self):
        ## Annule gradient
        #TODO
        self.gradientW = None
        self.gradientb = None"""

    def forward(self, X):
        ## Calcule la passe forward
        self.input = X
        return X@self.W + self.b

    def update_parameters(self, gradient_step=1e-3):
        ## Calcule la mise a jour des parametres selon le gradient calcule et le pas de gradient_step
        self.W -= gradient_step*self.gradientW
        self.b -= gradient_step*self.gradientb
        self._parameters = [self.W, self.b]

    def backward_update_gradient(self, input, delta):
        ## Calcule le gradient du coût par rapport aux paramètres et l’additionne au gradient (gradientW et gradientb)
        # - en fonction de l’entrée input et des δ de la couche suivante delta
        # Fait une moyenne sur les échantillons
        N = len(input)
        self.gradientW = input.T@stretch_vect(delta, N)/N
        self.gradientb = delta

    def backward_delta(self, input, delta):
        ## Calcule le gradient du coût par rapport aux entrées en fonction de l’entrée "input" et des deltas de la couche suivante "delta"
        return delta@self.X.T




## Tests module linéaire

In [ ]:
ml = LinearModule()

# 2. Mon second est ... non-linéaire !

In [33]:
class TanH(Module):

    def __init__(self):
        Module.__init__(self)
        #self.b = stretch_vect(np.random.rand(1, d), N) #TODO y a bien un biais

    """def zero_grad(self):
        ## Annule gradient
        #TODO
        self.gradientW = None
        self.gradientb = None"""

    def forward(self, X):
        ## Calcule la passe forward
        self.input = X
        return np.tanh(X) #+self.b

    def update_parameters(self, gradient_step=1e-3):
        ## il n'y a pas de paramètres à mettre a jour
        pass #TODO
        

    def backward_update_gradient(self, input, delta):
        pass

    def backward_delta(self, input, delta):
        ## Calcule le gradient du coût par rapport aux entrées en fonction de l’entrée "input" et des deltas de la couche suivante "delta"
        return prod_mat(stretch_vect(delta, len(input)),np.pow(1-np.tanh(input), 2))
    
class Sigmoide(Module):

    def __init__(self):
        Module.__init__(self)
        #self.b = stretch_vect(np.random.rand(1, d), N)

    """def zero_grad(self):
        ## Annule gradient
        #TODO
        self.gradientW = None
        self.gradientb = None"""

    def forward(self, X):
        ## Calcule la passe forward
        self.input = X
        return 1/(1 + np.exp(-X)) #+self.b #il faut que X soit un np array (-X)

    def update_parameters(self, gradient_step=1e-3):
        ## il n'y a pas de paramètres à mettre a jour
        pass #TODO
        

    def backward_update_gradient(self, input, delta):
        pass

    def backward_delta(self, input, delta):
        ## Calcule le gradient du coût par rapport aux entrées en fonction de l’entrée "input" et des deltas de la couche suivante "delta"
        return prod_mat(stretch_vect(delta, len(input)),div_mat(np.exp(-input), 1+np.exp(-input)))

In [ ]:
class Sequentiel(object):
    def __init__(self):
        self.layers = []
    
    def add(self, layer):
        self.layers.append(layer)

class Optim(object):
    def __init__(self, net, loss, eps):
        self.net = net
        self.loss = loss
        self.eps = eps
    
    def step(self, batch_x, batch_y):
        output = batch_x
        for layer in self.net.layers:
            output = layer.forward(output)
        cost = self.loss.forward(batch_y, output)
        delta = self.loss.backward(batch_y, output)
        for layer in reversed(self.net.layers):
            layer.backward_update_gradient(layer.input, delta)
            error = layer.backward_delta(layer.input, error)
        #TODO quand est ce qu on update les parametres ?
        for layer in self.net.layers:
            layer.update_parameters(gradient_step=self.eps)

